# F1 data analysis notebook 
#### @kevinofsydney

Notes:
- The 'KeyError' is because the name of the column gets consumed when it is turned into an index 
- df.describe() is useful
- oddsportal.com
- race_df['name'].unique()
- http://www.f1-predictor.com/category/data-science/
- https://www.f1hotornot.com/ - this is a very good form rating, and could be used to also indicate whether a driver is expected to overperform or underperform their average

### To do list
- Automate adding the column labels
- Calculate the average race result for each driver
- Create a working prototype to predict qualifier results
- Create a working prototype to predict race results

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

## Preprocessing

In [2]:
# Load the files
cir_df = pd.read_csv('circuits_l.csv')
conresult_df = pd.read_csv('constructor_results_l.csv')
constanding_df = pd.read_csv('constructor_standings_l.csv')
con_df = pd.read_csv('constructors_l.csv')
driver_df = pd.read_csv('driver_l.csv')
laptime_df = pd.read_csv('lap_times_l.csv')
quali_df = pd.read_csv('qualifying_l.csv')
race_df = pd.read_csv('races_l.csv')
result_df = pd.read_csv('results_l.csv')
status_df = pd.read_csv('status_l.csv')

# Set the correct indices 
race_df['date'] = pd.to_datetime(race_df['date'], dayfirst=True)

# driver_df.set_index('driverId', inplace=True)
quali_df.set_index('qualifyId', inplace=True)
laptime_df.set_index('raceId', inplace=True)
race_df.set_index('date', inplace=True)
result_df.set_index('resultId', inplace=True)

# race_df.sort_values('raceId', inplace=True)
race_df.sort_index(inplace=True)

# Remove unnecessary columns, or columns I still need to implement processing for
result_df = result_df.drop('number', axis=1)

resultsm_df = result_df[['raceId', 'driverId', 'constructorId', 'grid', 'positionOrder']]

def label_points(row):
    if row['positionOrder'] <= 3:
        return 1 #'top3'
    elif row['positionOrder'] > 3 and row['positionOrder'] <= 6:
        return 2 # 'top6'
    elif row['positionOrder'] > 6 and row['positionOrder'] <= 10:
        return 3 #'top10'
    else:
        return 4 #'no_points'

resultsm_df = resultsm_df.merge(race_df[['raceId', 'circuitId']], how='left', on=['raceId'])
resultsm_df['raceresult'] = resultsm_df.apply(label_points, axis=1)
resultsm_df.head(5)

,raceId,driverId,constructorId,grid,positionOrder,circuitId,raceresult
0,900,3,131,3,1,1,1
1,900,825,1,4,2,1,1
2,900,18,1,10,3,1,1
3,900,4,6,5,4,1,2
4,900,822,3,15,5,1,2


In [12]:
# Converting the constructors to strings, and then applying OHE. 
# Converting constructor names to strings allows for the columns to be named informatively
resultsm_df['constructorId'] = pd.Categorical(resultsm_df['constructorId'])
constructorDummies = pd.get_dummies(resultsm_df['constructorId'], prefix='constructor')
constructorDummies.head(5)

def name_constructors(row):
    if row['constructorId'] == 1:
        return 'MCL'
    elif row['constructorId'] == 3:
        return 'WIL'
    elif row['constructorId'] == 4:
        return 'REN'
    elif row['constructorId'] == 5:
        return 'TOR'
    elif row['constructorId'] == 6:
        return 'FER'
    elif row['constructorId'] == 9:
        return 'RBR'
    elif row['constructorId'] == 10:
        return 'RCP'
    elif row['constructorId'] == 15:
        return 'SAU'
    elif row['constructorId'] == 51:
        return 'ALF'
    elif row['constructorId'] == 131:
        return 'MER'
    elif row['constructorId'] == 206:
        return 'RUS'
    elif row['constructorId'] == 207:
        return 'CAT'
    elif row['constructorId'] == 208:
        return 'LOT'
    elif row['constructorId'] == 209:
        return 'MAN'
    elif row['constructorId'] == 210:
        return 'HAA'
    elif row['constructorId'] == 211:
        return 'RCP'
    
# resultsm_df = resultsm_df.drop(columns=['constructorID'])
resultsm_df['constructorIdString'] = resultsm_df.apply(name_constructors, axis=1)
resultsm_df['constructorIdString'] = pd.Categorical(resultsm_df['constructorIdString'])
cstrStringDummies = pd.get_dummies(resultsm_df['constructorIdString'], prefix='cstr')
cstrStringDummies.head(10)
# resultsm_df.head(5)

# Add the dummy columns back into the main results df 
resultsmx_df = pd.merge(resultsm_df, cstrStringDummies, left_index=True, right_index=True)

if 'constructorId' in resultsmx_df.columns:
    resultsmx_df = resultsmx_df.drop(columns=['constructorId'])

if 'constructorIdString' in resultsmx_df.columns: 
    resultsmx_df = resultsmx_df.drop(columns=['constructorIdString'])
    
resultsmx_df.head(5)

,raceId,driverId,grid,positionOrder,circuitId,raceresult,cstr_ALF,cstr_CAT,cstr_FER,cstr_HAA,...,cstr_MAN,cstr_MCL,cstr_MER,cstr_RBR,cstr_RCP,cstr_REN,cstr_RUS,cstr_SAU,cstr_TOR,cstr_WIL
0,900,3,3,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,900,825,4,2,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,900,18,10,3,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,900,4,5,4,1,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,900,822,15,5,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Modelling

In [69]:
# Splitting into training and test sets
# data - resultsm_df[['circuitId', 'driverId', 'constructorId', 'grid']]
# target - resultsm_df[['raceresult']]

from sklearn.model_selection import train_test_split

# Driver-OHE version
data_train, data_test, label_train, label_test = train_test_split(resultsmx_df.drop(columns=['raceresult']),
                                                                  resultsmx_df[['raceresult']],
                                                                  test_size=0.25, 
                                                                  random_state=1)

# data_train.describe()
# data_test.describe()
# label_train.describe()
label_test.describe()

,raceresult
count,577.000000
mean,3.079723
std,1.124485
min,1.000000
25%,2.000000
50%,4.000000
75%,4.000000
max,4.000000


In [39]:
# SVM
# Accuracy before OHE: 0.6308

from sklearn import svm
clf_svm = svm.SVC(kernel='linear', C=1).fit(data_train, label_train)
print("SVM accuracy:", clf_svm.score(data_test, label_test))

SVM accuracy: 1.0


C:\Users\kgatdula\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [70]:
# Naive-Bayes
# Accuracy before OHE: 0.6066

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gnb = GaussianNB()
clf_nb = gnb.fit(data_train, label_train)
pred = gnb.fit(data_train, label_train).predict(data_test)

# Print the accuracy score of the model
print("Naive-Bayes accuracy : ", accuracy_score(label_test, pred, normalize = True))

f = pd.DataFrame(data=pred)
f.columns = ['preds']
hello = pd.merge(data_test, f, left_index=True, right_index=True)
hello.to_csv('wassup.csv')

Naive-Bayes accuracy :  0.7660311958405546


C:\Users\kgatdula\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kgatdula\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


PermissionError: [Errno 13] Permission denied: 'wassup.csv'

## Code Graveyard

In [ ]:
# Find a driverId via their code
code = 'RAI'
did = driver_df[ driver_df['code'] == code]['driverId']
print("INFO: driverID for", code, "is:", int(did))

### Return the result for a single driver
resultsm_df[resultsm_df['driverId'] == int(did)].head(20)

resultsm_df['driverId'].nunique()

In [ ]:
# Merging the quali, driver_details and results dataframes together
quali_rdf = quali_df[['raceId', 'driverId', 'q1', 'q2', 'q3']]
master = driver_df[['driverId', 'code']].merge(quali_rdf, how='inner', on=['driverId'])
master = master.merge(result_df, how='inner', left_on=['raceId', 'driverId'], right_on=['raceId', 'driverId'])
# master.sort_values(by=['raceId', 'positionOrder']).head(50)
master.head(5)
# Train the model

In [ ]:
# Replace raceId with circuitId
from pandas.plotting import scatter_matrix
scatter_matrix(resultsm_df[['raceId','driverId','constructorId','grid','positionOrder','circuitId', 'raceresult']], figsize=(10,10))